### Prelude:

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
import random
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col

In [2]:
# import data
data = pd.read_csv("./../Datasets/ssi-data-cleaned.csv")

In [3]:
# Set up
treatments = {0: "No framing",
              1: "Negative science",
              2: "Religious",
              3: "Equity",
              4: "Efficiency",
              5: "Secular"}
data["treatment_frame"] = data["treatment_value"].map(treatments)

set_seed = 42
num_folds = 5
covariates_pre = ['gastax', 'carbtax', 'treaty', 'regcarb']

outcome_var = 'post_test'
covariates = ['age', 'party_id', 'employment_id', 'race_white', 'income_id', 
              'relationshipNum', 'educationNum', 'genderNum', 'prosociality', 'gastax', 
              'carbtax', 'treaty', 'regcarb', 'political_views', 'scientificconfidence', 
              'rewardconsequence', 'religiosity', 'economic_reasoning']
treatment_vars = [f"treatment_{i}" for i in range(1, 6)]
control_var = 'pre_test'

In [4]:
# Distribution of subjects across treatment conditions (like Table 1 from paper)
treatment_freq = data[["treatment_value", "treatment_frame"]].value_counts()
treatment_rel_freq = data["treatment_frame"].value_counts(normalize=True)
treatment_freq.to_frame().sort_index().join(treatment_rel_freq)

,,count,proportion
treatment_value,treatment_frame,,
0,No framing,22,0.135802
1,Negative science,23,0.141975
2,Religious,20,0.123457
3,Equity,34,0.209877
4,Efficiency,30,0.185185
5,Secular,33,0.203704


### Means tables:

In [5]:
pd.pivot_table(data, values=['pre_test', 'post_test', ],
               index=['treatment_value','treatment_frame'],
               aggfunc=['mean'])

mean          
                                 post_test  pre_test
treatment_value treatment_frame                     
0               No framing        6.454545  6.272727
1               Negative science  7.217391  6.695652
2               Religious         7.300000  6.473684
3               Equity            7.441176  7.029412
4               Efficiency        7.666667  7.533333
5               Secular           7.666667  7.484848

#### By political party (-1 id Dem, 1 is Rep, 0 is independent)

In [6]:
pd.pivot_table(data, values=["post_test", 'pre_test'],
               index=["party", "treatment_frame"], aggfunc=['mean'])

mean          
                       post_test  pre_test
party treatment_frame                     
-1    Efficiency        8.125000  8.125000
      Equity            7.916667  7.583333
      Negative science  9.083333  8.583333
      No framing        7.642857  7.214286
      Religious         9.846154  8.833333
      Secular           8.789474  8.421053
 0    Efficiency        8.000000  8.200000
      Equity            7.000000  7.000000
      Negative science  3.250000  3.000000
      No framing        5.200000  5.000000
      Religious         3.500000  3.000000
      Secular           7.857143  7.285714
 1    Efficiency        6.666667  6.111111
      Equity            5.833333  4.833333
      Negative science  6.285714  5.571429
      No framing        3.000000  4.000000
      Religious         2.200000  2.200000
      Secular           4.428571  5.142857

### Functions

#### Lin estimators

In [7]:
# with multiple treatments
def lin_estimator_mult_treat_formula(data, y_var, treatment_vars, covariates):
    """
    Inputs:
        data: pandas dataframe containing all x and y columns
        y_var: name of y variable
        treatment_vars: treatment dummy variables
        covariates: list of string names of covariate

    Returns: Lin estimator model, formula
    """
    # Demean the covariates
    for cov in covariates:
        # ignore binary variables
        if (data[cov].max() == 1 and data[cov].max() == 0) :
            data[cov + '_demeaned'] = data[cov]
        else:
            data[cov + '_demeaned'] = data[cov].dropna() - data[cov].dropna().mean()

    # Define the regression formula
    # Include each treatment indicator
    treatments_formula = " + ".join(treatment_vars)

    # Include each interaction term (automatically includes individual covariates)
    interactions = []
    for treatment in treatment_vars:
        for cov in covariates:
            interactions.append(f"{cov+ '_demeaned'} * {treatment}")
    
    interactions_formula = " + ".join(interactions)

    # Full formula -- include any other control(s)
    formula = f"{y_var} ~ {treatments_formula} + {interactions_formula}"

    # Fit the regression model and save results object
    model = sm.OLS.from_formula(formula, data=data).fit()

    # Return results object with robust covariance type
    return model.get_robustcov_results(cov_type="HC3"), formula

# with one treatment
def lin_estimator_formula(data, y_var, treatment_var, covariates):
    """
    Inputs:
        data: pandas dataframe containing all x and y columns
        y_var: name of y variable
        treatment_var: single treatment variable 
        covariates: list of string names of covariate

    Returns: Lin estimator model, formula
    """
    # Demean the covariates
    for cov in covariates:
        # ignore binary variables
        if (data[cov].max() == 1 and data[cov].max() == 0) :
            data[cov + '_demeaned'] = data[cov]
        else:
            data[cov + '_demeaned'] = data[cov].dropna() - data[cov].dropna().mean()

    # Define the regression formula

    # Include each interaction term (automatically includes individual covariates)
    interactions = []
    for cov in covariates:
        interactions.append(f"{cov+ '_demeaned'} + {cov+ '_demeaned'} * {treatment_var}")
    
    interactions_formula = " + ".join(interactions)

    # Full formula -- include any other control(s)
    formula = f"{y_var} ~ {treatment_var} + {interactions_formula}"

    # Fit the regression model and save results object
    model = sm.OLS.from_formula(formula, data=data).fit()

    # Return results object with robust covariance type
    return model.get_robustcov_results(cov_type="HC3"), formula

#### Other estimation helpers

In [8]:
# Function to extract treatment effects from model
def extract_treatment_effects(model, treatment_vars):
    coefs = dict(zip(model.model.exog_names, model.params))
    effects = {var: coefs.get(var, 0) if coefs.get(var) is not None else 0 for var in treatment_vars}
    return effects

# Function to find the best treatment
def find_best_treatment(effects):
    if effects:
        return max(effects, key=effects.get)
    return None

# Function to assign the best treatment indicator
def assign_best_treatment_indicator(test_data, best_treatment):
    if best_treatment:
        test_data['best_treatment_indicator'] = (test_data[best_treatment] == 1).astype(int)
        test_data['not_best_treatment_indicator'] = ((test_data[best_treatment] != 1) * (test_data['treatment_value']!=0) ).astype(int)
    else:
        data['best_treatment_indicator'] = 0

# Train models and predict outcomes
def train_and_predict(train_data, test_data, features, random_state):
    predictions = {}

    # Train a model for each treatment condition and predict for the test data
    for treatment in treatment_vars:
        # Assuming binary treatment, filter data where treatment is active
        treated_data = train_data[train_data[treatment] == 1]
        rf = RandomForestRegressor(n_estimators=100, random_state=42)
        rf.fit(treated_data[features], treated_data[outcome_var])
        # Store predictions for each treatment
        predictions[treatment] = rf.predict(test_data[features])

    return predictions

# Function to check if the treatment equals 1
def check_treatment(row, column):
    treatment_col = row[column]
    return int(row[treatment_col] == 1)

# Function to assing the best personalized treatment indicator
def assign_best_personalized_treatment_indicator(test_data, results):
    # Create a DataFrame from the results with appropriate indexing
    results_df = pd.DataFrame(results, index=test_data.index)
    # Use np.argmax on axis=1 to find the indices of maximum values along the horizontal axis
    best_treatment_indices = np.argmax(results_df.values, axis=1)
    # Convert indices to a Series to use the map function
    best_treatment_series = pd.Series(best_treatment_indices, index=test_data.index)
    # Map indices to treatment variable names
    best_treatment = best_treatment_series.map(dict(enumerate(treatment_vars)))
    test_data['best_personalized_treatment'] = best_treatment
    # Apply the function across the DataFrame rows
    test_data['best_personalized_treatment_indicator'] = test_data.apply(check_treatment, column = 'best_personalized_treatment', axis=1)
    test_data['not_best_personalized_treatment_indicator'] = ((test_data['best_personalized_treatment_indicator'] != 1) * (test_data['treatment_value']!=0) ).astype(int)


#### Difference in means

In [9]:
# Simple difference in means estimator
treatments_formula = " + ".join(treatment_vars)
formula = f"post_test ~ {treatments_formula}"

# Fit the regression model and save results object
model = sm.OLS.from_formula(formula, data=data).fit()

model0_results = model.get_robustcov_results(cov_type="HC3")
model0_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              post_test   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                 -0.016
Method:                 Least Squares   F-statistic:                    0.4947
Date:                Mon, 29 Apr 2024   Prob (F-statistic):              0.780
Time:                        18:49:40   Log-Likelihood:                -412.80
No. Observations:                 162   AIC:                             837.6
Df Residuals:                     156   BIC:                             856.1
Df Model:                           5                                         
Covariance Type:                  HC3                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       6.4545      0.696      9.278      0.000       5.080       7.829
treatment_1     0.7628      1.010      0.755      0.451      -1.233       2.759
treatment_2     0.8455      1.204      0.702      0.483      -1.532       3.223
treatment_3     0.9866      0.828      1.191      0.236      -0.650       2.623
treatment_4     1.2121      0.840      1.443      0.151      -0.448       2.872
treatment_5     1.2121      0.893      1.357      0.177      -0.552       2.976
==============================================================================
Omnibus:                       11.349   Durbin-Watson:                   1.903
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               12.451
Skew:                          -0.676   Prob(JB):                      0.00198
Kurtosis:                       2.867   Cond. No.                         7.57
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [12]:
# Lin estimator
model1_results, model1_formula = lin_estimator_mult_treat_formula(data,
                                                          "post_test",
                                                          treatment_vars,
                                                          covariates)

model1_pre_results, model1_formula = lin_estimator_mult_treat_formula(data,
                                                          "post_test",
                                                          treatment_vars,
                                                          covariates_pre)
# model1_results.summary()

In [13]:
print (summary_col([model0_results, model1_results, model1_pre_results],stars=True,float_format='%0.3f',
                  model_names=['Difference-in-means\n(1)','Lin (all covariates)\n(2)','Lin (pre-test only)\n(3)'],
                  info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                             'R2':lambda x: "{:.2f}".format(x.rsquared)},
                  regressor_order=['Intercept'] + [f"treatment_{i}" for i in range(1, 6)]
                  drop_omitted=True))

SyntaxError: invalid syntax. Perhaps you forgot a comma? (852086519.py, line 5)

In [16]:
model1_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              post_test   R-squared:                       0.966
Model:                            OLS   Adj. R-squared:                  0.886
Method:                 Least Squares   F-statistic:                     13.05
Date:                Mon, 29 Apr 2024   Prob (F-statistic):           2.30e-17
Time:                        18:51:21   Log-Likelihood:                -140.01
No. Observations:                 162   AIC:                             508.0
Df Residuals:                      48   BIC:                             860.0
Df Model:                         113                                         
Covariance Type:                  HC3                                         
=============================================================================================================
                                                coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
Intercept                                     6.6743      2.292      2.912      0.005       2.065      11.283
treatment_1                                   0.5908      2.612      0.226      0.822      -4.661       5.842
treatment_2                                   0.9344      6.206      0.151      0.881     -11.544      13.413
treatment_3                                   0.5894      2.315      0.255      0.800      -4.066       5.245
treatment_4                                   0.4550      2.336      0.195      0.846      -4.243       5.153
treatment_5                                   0.8329      2.351      0.354      0.725      -3.894       5.559
age_demeaned                                  0.1634      0.205      0.797      0.429      -0.249       0.575
age_demeaned:treatment_1                     -0.1900      0.290     -0.655      0.516      -0.774       0.394
party_id_demeaned                             0.2730      0.924      0.295      0.769      -1.585       2.131
party_id_demeaned:treatment_1                -0.6374      1.709     -0.373      0.711      -4.073       2.798
employment_id_demeaned                       -1.6619      5.495     -0.302      0.764     -12.711       9.387
employment_id_demeaned:treatment_1            2.5168      7.249      0.347      0.730     -12.058      17.092
race_white_demeaned                          -1.6626      2.182     -0.762      0.450      -6.051       2.725
race_white_demeaned:treatment_1               1.8290      4.209      0.435      0.666      -6.634      10.292
income_id_demeaned                            0.1904      0.500      0.381      0.705      -0.815       1.195
income_id_demeaned:treatment_1               -0.1278      0.607     -0.211      0.834      -1.348       1.092
relationshipNum_demeaned                      0.3899      3.993      0.098      0.923      -7.639       8.419
relationshipNum_demeaned:treatment_1         -1.1128      6.062     -0.184      0.855     -13.302      11.076
educationNum_demeaned                         1.2341      4.828      0.256      0.799      -8.473      10.941
educationNum_demeaned:treatment_1            -1.0038      5.772     -0.174      0.863     -12.609      10.601
genderNum_demeaned                           -1.5496      3.831     -0.404      0.688      -9.252       6.153
genderNum_demeaned:treatment_1                1.0536      5.067      0.208      0.836      -9.135      11.242
prosociality_demeaned                         0.1728      2.164      0.080      0.937      -4.178       4.524
prosociality_demeaned:treatment_1            -0.3816      3.698     -0.103      0.918      -7.816       7.053
gastax_demeaned                               0.8699      2.072      0.420      0.676      -3.296       5.036
gas

### Cross-Validation Regression:

In [ ]:
# Shuffle data and split into folds
shuffled = data.sample(frac=1, random_state=set_seed)
folds = np.array_split(shuffled, num_folds)

# Initialize storage for results and effects
combined_data = data.iloc[:0,:].copy()
all_effects = []
best_treatments = []

# Iterate over each fold, using it as the test set, and the others as the training set
for i in range(num_folds):
    test_fold = folds[i]
    training_folds = pd.concat([folds[j] for j in range(num_folds) if j != i])
    
    # Train model on the combined training folds
    training_model = lin_estimator_mult_treat_formula(training_folds, outcome_var, treatment_vars, covariates_pre)[0]
    training_effects = extract_treatment_effects(training_model, treatment_vars)
    
    # Find the best treatment from the training model
    best_treatment = find_best_treatment(training_effects)
    assign_best_treatment_indicator(test_fold, best_treatment)

    all_effects = all_effects + [training_effects]
    best_treatments = best_treatments + [best_treatment]
    combined_data = pd.concat([combined_data, test_fold])

# Simple difference in means estimator
treatments_formula = " + ".join(['best_treatment_indicator', 
                                                   'not_best_treatment_indicator'])
formula = f"post_test ~ {treatments_formula}"

# Fit the regression model and save results object
model = sm.OLS.from_formula(formula, data=combined_data).fit()

model3_results = model.get_robustcov_results(cov_type="HC3")

model3_pre_results = lin_estimator_mult_treat_formula(combined_data, 
                                                  outcome_var, 
                                                  # we include `not_best_personalized_treatment_indicator` so that the 
                                                  # `best_personalized_treatment_indicator` is compared to control only
                                                  ['best_treatment_indicator', 
                                                   'not_best_treatment_indicator'], 
                                                  covariates_pre)[0]

#### Best treatments

In [ ]:
best_treatments

### Random Forest:

In [ ]:
# Shuffle data and split into folds
shuffled = combined_data.sample(frac=1, random_state=set_seed).copy()
folds = np.array_split(shuffled, num_folds)

features = covariates

# Initialize storage for results and effects
combined_data = data.iloc[:0,:].copy()
all_effects = []
best_treatments = []

# Iterate over each fold, using it as the test set, and the others as the training set
for i in range(num_folds):
    test_fold = folds[i]
    training_folds = pd.concat([folds[j] for j in range(num_folds) if j != i])
    
    # Train model on the combined training folds, predict on test data
    test_results = train_and_predict(training_folds, test_fold, features, set_seed)
    assign_best_personalized_treatment_indicator(test_fold, test_results)
    for fold_number in range(0, num_folds):
        test_fold[f'fold_{fold_number}'] = 0
    test_fold[f'fold_{i}'] = 1

    combined_data = pd.concat([combined_data, test_fold])

# Simple difference in means estimator
treatments_formula = " + ".join(['best_personalized_treatment_indicator', 
                                                   'not_best_personalized_treatment_indicator'])
formula = f"post_test ~ {treatments_formula}"

# Fit the regression model and save results object
model = sm.OLS.from_formula(formula, data=combined_data).fit()

model4_results = model.get_robustcov_results(cov_type="HC3")

model4_pre_results = lin_estimator_mult_treat_formula(combined_data, 
                                                  outcome_var, 
                                                  # we include `not_best_personalized_treatment_indicator` so that the 
                                                  # `best_personalized_treatment_indicator` is compared to control only
                                                  ['best_personalized_treatment_indicator', 
                                                   'not_best_personalized_treatment_indicator'], 
                                                  covariates_pre)[0]

In [ ]:
print (summary_col([model3_results, model4_results],stars=True,float_format='%0.3f',
                  model_names=['Best fixed arm\n(1)', 'Best personalized arm\n(2)'],
                  info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                             'R2':lambda x: "{:.2f}".format(x.rsquared)},
                  regressor_order=['Intercept', 'best_treatment_indicator', 'best_personalized_treatment_indicator'],
                  drop_omitted=True))

In [ ]:
# check that approximately 1/6 of people were assigned the best treatment and best personalized treatment
combined_data['best_treatment_indicator'].mean()

In [ ]:
combined_data['best_personalized_treatment_indicator'].mean()